![Python packaging: Lo estás haciendo mal](img/cover.jpeg)

## 2021-03-04 @ Python Madrid

# Resumen

1. Introducción
2. Brevísima historia de una pesadilla
3. Cómo instalar paquetes
4. Cómo especificar dependencias para aplicaciones
5. Cómo distribuir código
6. Conclusiones y preguntas

### Preguntas hechas con antelación en el Meetup

---

Casos de uso para:

- Installation: python setup.py install
- Software dist: python setup.py sdist
- Binary dist: python setup.py bdist

---

Compatibilidad poetry vs buildpacks, gracias!

https://github.com/heroku/heroku-buildpack-python/issues/796

---

¿Por qué todavía seguimos diferenciando entre librería y aplicación?
¿Cuándo estará disponible para su uso general lo especificado en el PEP631?
¿Qué es un Platypus?

---

Cómo referenciar paquetes privados para que puedan instalarse con pip en distintos entornos de CI/CD

---

Integración y despliegue continuo, cómo organizarlo en grandes empresas, artefactory, Jenkins, etc. convivencia con otros proyectos, puesta en producción.

---

Instalación para Alpine linux mini root fs para containers!

---

* tips para pasar un poyecto legacy en pip a poetry (requirements a toml).
* Manejo de multilenguaje y compilado de diccionarios en la instalación

---

Como instalar una libreria de manera agnostica con el host?, me refiero a instalar una libreria en windows pero con arquitectura x86 linux.

Ya que tuve un proyecto y la unica solucion fue descargar una ISO de un ubuntu server x86 y ahi hacer el pip install (en ese caso fue PARAMIKO) ya que donde yo ejecutaba mi Script tenian ese SO y yo desarrollaba en una MAC y me marcaba error por los HEADERS del OS.

# ¿Quién es este?

<img src="img/yo.jpg" alt="¡Yo!" style="width: 35%; float: right" />

<ul style="width: 55%">
    <li><strong>Ingeniero aeroespacial</strong> con pasión por las órbitas 🛰</li>
    <li>Ex presidente y fundador de <strong>Python España</strong> y ex organizador de <strong>PyCon España</strong> y <strong>Python Madrid</strong> 🐍</li>
    <li><strong>Mission Planning &nbsp; Execution Engineer</strong> en <strong>Satellogic</strong> 🌍</li>
    <li><em>I code like a girl 🌈</em></li>
    <li>Activista del Conocimiento Libre y la Ciencia Abierta 🄯</li>
    <li>Melómano 🎸</li>
</ul>

Sígueme! https://github.com/astrojuanlu/

<div style="float: clear"></div>

# Una aclaración importante

![Paternalista](img/paternalista.jpg)

- "Lo estás haciendo mal" es un título condescendiente a propósito a modo de broma
- Hay valor en tener unas buenas prácticas comunes: reducir la fricción entre proyectos y poder dedicarnos a lo importante

# Brevísima historia de una pesadilla

## O «cómo aprendimos a odiar pip»

![Jannis Leidel, 2009](img/jezdez2009.png)

![Martin Natano, 2014](img/martinnatano2014.png)

!["Una abominación"](img/fperez_org2012.png)

### 2013

![¡Camaradas!](img/python_comrades-orig.png)

!["Malware"](img/dwf2015.png)

### ???

![¿Camaradas...?](img/python_comrades-edit.png)

https://www.slideshare.net/misterwang/pydata-past-present-future-pydata-sv-2014-keynote

![Vamos, un lío](img/this-packaging-problem.jpg)

![Empecemos de nuevo](img/shaking-head-2.gif)

# Cómo instalar paquetes

![Nivel básico](img/charmander.webp)

✨✨✨ _Instale el paquete deseado en dos sencillos pasos:_ ✨✨✨

```
$ source .venv/bin/activate
(.venv) $ python -m pip install ipython
```

> «Sí, pero... ¡es que yo lo quiero en uno!»

Mal menor: pip nuevo ⚠️⚠️⚠️

```
$ pip install ipython  # Hmmm...
```

Por defecto se instala en `${HOME}/.local/lib` (a escondidas se ha usado `--user`). ¿Queremos eso?

```
In [1]: import sys

In [2]: sys.prefix
Out[2]: '/usr'

In [3]: import urllib3

In [4]: urllib3.__file__
Out[4]: '/usr/lib/python3/dist-packages/urllib3/__init__.py'

In [3]: import IPython

In [4]: IPython.__file__
Out[4]: '/home/juanlu/.local/lib/python3.8/site-packages/IPython/__init__.py'
```

Ya tenemos dos localizaciones distintas 😨

> «Sí, pero... ¡es que yo lo quiero en uno!»

Catástrofe en ciernes: pip viejo 🚨🚨🚨

```
$ pip install urllib3  # Hmmm...
```

<pre><code style="color: #c00">[Errno 13] Permission denied: '/usr/lib/python3.8'</code></pre>

```
$ sudo pip install urllib3  # NOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO
```

![Catástrofe](img/nuclear-explosion.gif)

- Hacer `sudo pip install` 🚫 puede desestabilizar dependencias críticas de tu sistema
  - Nótese que en Windows esto no es un problema
- Para desarrollo, **nunca jamás se usa el Python del sistema**
- ¡Siempre utilizar un **entorno virtual**!

Opciones:

- `venv` https://docs.python.org/3/library/venv.html ¡en la biblioteca estándar desde Python 3.3 (2012)!
- `virtualenv` https://pypi.org/project/virtualenv/, más rápido y flexible ([recientemente resucitado por Bernát Gábor](https://discuss.python.org/t/virtualenv-20-0-0-beta1-is-available/3077?u=astrojuanlu))

Otras opciones:

- `virtualenvwrapper` no tiene actividad desde febrero de 2019 😓 (¡aunque aún funciona!)
- `pyenv` rompe el `$PATH` y algunos scripts 😓 https://github.com/pyenv/pyenv/issues/1112
- `Pipenv` [trata de hacer demasiadas cosas](https://hynek.me/articles/python-app-deps-2018/#pipenv), [avanza con mucho esfuerzo](https://discuss.python.org/t/work-towards-next-pipenv-release/3781?u=astrojuanlu), y [tiene ciertos problemas](https://chriswarrick.com/blog/2018/07/17/pipenv-promises-a-lot-delivers-very-little/)
- `Poetry` nunca lo usé 🤔

> ¿¿Y qué pasa con Alpine??

```
$ docker run -it --rm --name pip-wheel python:3.8-alpine sh
/ # python -m pip install numpy
Collecting numpy
  Downloading numpy-1.20.1.zip (7.8 MB)
     |████████████████████████████████| 7.8 MB 10.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... error
```

<pre><code style="color: #c00">
    ERROR: Command errored out with exit status 1:
     command: /usr/local/bin/python /usr/local/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py prepare_metadata_for_build_wheel /tmp/tmpzz6m5yah
         cwd: /tmp/pip-install-9mlwuba9/numpy
    Complete output (226 lines):
...
      File "numpy/core/setup.py", line 676, in get_mathlib_info
        raise RuntimeError("Broken toolchain: cannot link a simple C program")
    RuntimeError: Broken toolchain: cannot link a simple C program
</code></pre>

🤯

- Normalmente, pip descarga un [_Source Distribution_ o "sdist"](https://packaging.python.org/glossary/#term-Source-Distribution-or-sdist), que contiene todo lo necesario para instalar el paquete
- Sin embargo, "todo lo necesario" a veces puede incluir código en C, C++, FORTRAN, Rust, JavaScript...
- Y por tanto, ¡**necesito herramientas _que no se instalan con pip_**!

- [El PEP 427 define el formato `wheel`](https://www.python.org/dev/peps/pep-0427/) que resuelve este problema ofreciendo un [_Built Distribution_](https://packaging.python.org/glossary/#term-Built-Distribution)
- ¡Simplemente extraer archivos! Más rápido de instalar, y ✨ no necesita nada más que pip ✨
- Sin embargo, ¡este formato no es multiplataforma!

El formato del nombre del archivo indica la compatibilidad:

<pre><code><span style="background-color: #fcc">{distribution}</span>-<span style="background-color: #ffc">{version}(-{build tag})?</span>-<span style="background-color: #cfc">{python tag}-{abi tag}</span>-<span style="background-color: #cff">{platform tag}.whl</span></code></pre>

- `distribution`: Nombre del paquete (`Django`, `numpy`)
- `version`: Versión (compatible con el [PEP 440](https://www.python.org/dev/peps/pep-0440/))
- `python tag` y `abi tag`: Implementación de Python (`cp38`, `pp372-pypy3_72`)
- `platform`: **Plataforma**

Por ejemplo:

```
numpy-1.20.1-cp38-cp38-manylinux1_x86_64.whl
```

..."many _what_"???

> For Linux, the situation is much more delicate.

[PEP 513 -- A Platform Tag for Portable Linux Built Distributions](https://www.python.org/dev/peps/pep-0513/)

- No hay "un" Linux, hay muchos
- Hay varias causas (explicadas en el PEP) de incompatibilidad entre binarios de Linux
- La más importante, _dependencia dinámica en diferentes versiones de la GNU C Library, `glibc`_
- El PEP 513 estandariza unas versiones mínimas, basadas en CentOS 5.11, y le da el nombre `manylinux1`
- Después vinieron `manylinux2010` (CentOS 6), `manylinux2014` (CentOS 7), y vendrán más

- ...pero es que Alpine **no usa `glibc`**, sino `musl`! 😭😭😭
- Hay un [pre-PEP para crear wheels para `musl`](https://discuss.python.org/t/pre-pep-platform-tag-for-linux-distributions-using-musl/7165?u=astrojuanlu), pero de momento hay que usar métodos alternativos

En resumen:

- `python -m pip install` + `venv` = ❤️
- pip está bien mantenido y está recibiendo muchas mejoras ❤️
- ¡Cuidado con los wheels!

# Cómo especificar dependencias para aplicaciones 

![Nivel intermedio](img/charmeleon.webp)


✨✨✨ _Especifique las dependencias deseadas en dos sencillos pasos:_ ✨✨✨

```
(.venv) $ echo "django<3" > requirements.in
(.venv) $ python -m pip install pip-tools && pip-compile
```

[pip-compile](https://pypi.org/project/pip-tools/) toma como entrada un archivo `requirements.in` y produce un `requirements.txt` con todas las versiones fijadas:

```
(.venv) $ cat requirements.txt
#
# This file is autogenerated by pip-compile
# To update, run:
#
#    pip-compile
#
django==2.2.19
    # via -r requirements.in
pytz==2021.1
    # via django
sqlparse==0.4.1
    # via django
```

Perfecto para tener un entorno reproducible 💯

Una vez fijadas, ¡por defecto no se actualizan! 

```
(.venv) $ echo "django" > requirements.in
(.venv) $ pip-compile
#
# This file is autogenerated by pip-compile
# To update, run:
#
#    pip-compile
#
django==2.2.19
    # via -r requirements.in
pytz==2021.1
    # via django
sqlparse==0.4.1
    # via django
```

Se acabaron los pipelines rotos por nuevas versiones ✨

Solo se actualizan si lo pedimos:

```
(.venv) $ pip-compile -P django
#
# This file is autogenerated by pip-compile
# To update, run:
#
#    pip-compile
#
asgiref==3.3.1
    # via django
django==3.1.7
    # via -r requirements.in
pytz==2021.1
    # via django
sqlparse==0.4.1
    # via django
```

Y, para poner nuestro entorno al día:

```
(.venv) $ pip-sync
Collecting asgiref==3.3.1
  Using cached asgiref-3.3.1-py3-none-any.whl (19 kB)
Collecting django==3.1.7
  Using cached Django-3.1.7-py3-none-any.whl (7.8 MB)
Collecting pytz==2021.1
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
Collecting sqlparse==0.4.1
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
Installing collected packages: sqlparse, pytz, asgiref, django
Successfully installed asgiref-3.3.1 django-3.1.7 pytz-2021.1 sqlparse-0.4.1
```

...que es casi lo mismo que hacer:

```
(.venv) $ python -m pip install -r requirements.txt 
Requirement already satisfied: asgiref==3.3.1 in ./.venv/lib/python3.8/site-packages (from -r requirements.txt (line 7)) (3.3.1)
Requirement already satisfied: django==3.1.7 in ./.venv/lib/python3.8/site-packages (from -r requirements.txt (line 9)) (3.1.7)
Requirement already satisfied: pytz==2021.1 in ./.venv/lib/python3.8/site-packages (from -r requirements.txt (line 11)) (2021.1)
Requirement already satisfied: sqlparse==0.4.1 in ./.venv/lib/python3.8/site-packages (from -r requirements.txt (line 13)) (0.4.1)
```

😍😍😍

Otras alternativas:

- Ya hemos hablado de `Pipenv`
- No puedo agregar mucho sobre `Poetry`
- ~`setup.py`~ ¡Calma! No hemos llegado aún, [y además no es exactamente para esto](https://caremad.io/posts/2013/07/setup-vs-requirement/)

En resumen:

- `pip-compile` + `pip-sync` (o simplemente `pip`) = ❤️
- _Todo el mundo_ entiende `requirements.txt`, ¡usémoslo!
- Básicamente, haced caso a Hynek https://hynek.me/articles/python-app-deps-2018/

# Cómo distribuir código 

![Nivel avanzado](img/charizard.webp)

- Varios enfoques. Lista de PEPs, un poco aburrido, mejor busquemos otra forma. Pero empecemos a enumerar.
  - PEP 518 https://www.python.org/dev/peps/pep-0518/ introducción de `pyproject.toml` para dependencias en tiempo de **instalación**
    - Sustituye a `setup_requires` de setuptools
      - Por cierto, `easy_install` is dead! https://twitter.com/codewithanthony/status/1356285880500043776
    - Muy buen artículo por Brett Cannon https://snarky.ca/clarifying-pep-518/
  - PEP 517 https://www.python.org/dev/peps/pep-0517/ especificación para nuevos build systems que no sean distutils o setuptools (flit, poetry)
  - PEP 632 https://www.python.org/dev/peps/pep-0632/ distutils está obsoleto! ahora hay que usar solamente setuptools
    - Lo seguirán necesitando por una temporada los que usen extensiones
  - PEP 621 https://www.python.org/dev/peps/pep-0621/ estandarización de `pyproject.toml` para unificar setuptools, flit, y Poetry!
    - Esto mejor contarlo al final del todo
    - Viene al caso de poetry y el buildpack de Heroku
    - https://twitter.com/brettsky/status/1366919850510049280
- Perfiles de gente:
  - El desarrollador que quiere publicar código
    - `setup.py` ha muerto!!
      - `python setup.py install` -> `pip install .`
      - `python setup.py develop` -> `pip install -e .`
      - `python setup.py sdist bdist_wheel` -> `pip install build && python -m build`
    - Usar repositorios internos: devpi, https://pypi.org/project/pypiserver/
    - No usar git!! Mucho más lento
    - Y además, ataques de supply chain
    - Mover la parte de los PEPs aquí?
    - PEP 621 - hace 5 días Thomas Kluyver (flit) abrió un PR!!!! https://github.com/takluyver/flit/pull/393
    - Uso de tox para tener paridad con CI! Y además ahora viene tox4
    - Cómo seguir: https://github.com/astrojuanlu/cookiecutter-pylib